In [1]:
# classifier for bacterial vs fungal

In [2]:
import os
import keras
import numpy as np
import cv2
np.random.seed(2020)

Using TensorFlow backend.


In [3]:
def dataset():
    X = []
    y = []
    labels = {line.strip().split(',')[0] : int(line.strip().split(',')[1]) for line in open('labels.csv')}
    for img in labels:
        if labels[img] != 2:
            img_arr = cv2.imread(os.path.join('images_preprocessed', img))
            X.append(img_arr)
            y.append(labels[img])
    
    X = np.asarray(X, dtype=np.float64)
    y = np.asarray(y)
    
    X -= np.mean(X)
    X /= np.std(X)
    
    return X, y

In [4]:
X, y = dataset()

In [5]:
print(X.shape, y.shape)

(10, 200, 300, 3) (10,)


In [6]:
from tensorflow.python.client import device_lib

def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']

print(get_available_gpus())

['/device:GPU:0', '/device:GPU:1', '/device:GPU:2']


In [7]:

from keras.models import Sequential
from keras.layers import TimeDistributed, GlobalAveragePooling2D, Activation, Dense, Input
from keras.layers import BatchNormalization, Conv2D, MaxPooling2D, AveragePooling2D, GlobalAveragePooling2D
from keras.layers import GlobalMaxPooling2D
from keras.layers.recurrent import LSTM
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from keras.layers import Dropout, Flatten
from keras.layers import concatenate
from keras import regularizers
from keras import initializers
from keras import constraints
from keras.models import Model
from keras import backend as K
from keras.utils.layer_utils import convert_all_kernels_in_model
from keras.utils.data_utils import get_file
from keras.utils import conv_utils
from keras.engine.topology import get_source_inputs
from keras.engine import InputSpec
from keras.applications.xception import Xception
from keras import metrics
from keras import layers
from keras.layers import Activation
from keras.layers import SeparableConv2D
from keras.models import load_model

In [8]:
#linear
def linear(dropout):
    inputs = Input(shape=(200, 300, 3))
    x = Flatten()(inputs)
    x = Dropout(dropout)(x)
    x = Dense(2, activation='softmax')(x)
    model = Model(inputs, x, name='linear')
    return model

In [9]:
def vanillacnn(dropout):
    inputs = Input(shape=(200, 300, 3))
    x = Conv2D(32, (3, 3), strides=(2, 2), padding='same')(inputs)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), padding='same')(x)
    x = Dropout(dropout)(x)
    
    x = Conv2D(64, (3, 3), strides=(2, 2), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), padding='same')(x)
    x = Dropout(dropout)(x)
    
    x = Conv2D(128, (3, 3), strides=(2, 2), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), padding='same')(x)
    x = Dropout(dropout)(x)
    
#     x = Conv2D(256, (3, 3), strides=(2, 2), padding='same')(x)
#     x = BatchNormalization()(x)
#     x = Activation('relu')(x)
#     x = MaxPooling2D((2, 2), strides=(2, 2), padding='same')(x)
    
    x = Flatten()(x)
    x = Dropout(dropout)(x)
    x = Dense(2, activation='softmax', name='predictions')(x)
    model = Model(inputs, x, name='xception')
    return model
    
    

In [12]:
correct = 0
for i in range(y.shape[0]):
    model_name = os.path.join('models', 'model2_linear_%s.h5' % (i))
    train_fold = np.asarray([j for j in range(y.shape[0]) if j != i])
    val_fold = np.asarray([i])
    model = linear(0.5)

    adam = keras.optimizers.Adam(lr=0.000001)
    model.compile(optimizer=adam, loss='sparse_categorical_crossentropy', metrics=[metrics.sparse_categorical_accuracy])


    checkpoint = ModelCheckpoint(model_name, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
    stop = EarlyStopping(patience=20, verbose=1)

    callbacks_list = [checkpoint, stop]

    model.summary()
    
    model.fit(x=X[train_fold], 
          y=y[train_fold], 
          batch_size=9, 
          epochs=1000, 
          verbose=1, 
          callbacks=callbacks_list, 
          validation_data=(X[val_fold], y[val_fold]), 
          shuffle=False)
    
    print('====eval=====')
    trained_model = load_model(model_name)
    print(trained_model.metrics_names)
    res = trained_model.evaluate(x=X[val_fold], y=y[val_fold])
    print(res)
    correct += res[1]
    print('====done=====')

print('Linear Model Leave-one-out Cross Validation Accuracy:', correct/y.shape[0])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_21 (InputLayer)        (None, 200, 300, 3)       0         
_________________________________________________________________
flatten_21 (Flatten)         (None, 180000)            0         
_________________________________________________________________
dropout_51 (Dropout)         (None, 180000)            0         
_________________________________________________________________
dense_11 (Dense)             (None, 2)                 360002    
Total params: 360,002
Trainable params: 360,002
Non-trainable params: 0
_________________________________________________________________
Train on 9 samples, validate on 1 samples
Epoch 1/1000
9/9 [==============================] - 4s 393ms/step - loss: 1.7885 - sparse_categorical_accuracy: 0.3333 - val_loss: 0.2147 - val_sparse_categorical_accuracy: 1.0000

Epoch 00001: val_loss improved from inf to 0.


Epoch 00003: val_loss did not improve from 0.11471
Epoch 4/1000
9/9 [==============================] - 0s 2ms/step - loss: 0.9286 - sparse_categorical_accuracy: 0.7778 - val_loss: 0.1161 - val_sparse_categorical_accuracy: 1.0000

Epoch 00004: val_loss did not improve from 0.11471
Epoch 5/1000
9/9 [==============================] - 0s 2ms/step - loss: 0.7865 - sparse_categorical_accuracy: 0.5556 - val_loss: 0.1174 - val_sparse_categorical_accuracy: 1.0000

Epoch 00005: val_loss did not improve from 0.11471
Epoch 6/1000
9/9 [==============================] - 0s 2ms/step - loss: 0.9676 - sparse_categorical_accuracy: 0.5556 - val_loss: 0.1169 - val_sparse_categorical_accuracy: 1.0000

Epoch 00006: val_loss did not improve from 0.11471
Epoch 7/1000
9/9 [==============================] - 0s 2ms/step - loss: 1.4252 - sparse_categorical_accuracy: 0.3333 - val_loss: 0.1196 - val_sparse_categorical_accuracy: 1.0000

Epoch 00007: val_loss did not improve from 0.11471
Epoch 8/1000
9/9 [==========


Epoch 00011: val_loss improved from 0.68773 to 0.66768, saving model to models/model2_linear_2.h5
Epoch 12/1000
9/9 [==============================] - 0s 2ms/step - loss: 0.6504 - sparse_categorical_accuracy: 0.5556 - val_loss: 0.6560 - val_sparse_categorical_accuracy: 1.0000

Epoch 00012: val_loss improved from 0.66768 to 0.65605, saving model to models/model2_linear_2.h5
Epoch 13/1000
9/9 [==============================] - 0s 2ms/step - loss: 0.7462 - sparse_categorical_accuracy: 0.5556 - val_loss: 0.6439 - val_sparse_categorical_accuracy: 1.0000

Epoch 00013: val_loss improved from 0.65605 to 0.64387, saving model to models/model2_linear_2.h5
Epoch 14/1000
9/9 [==============================] - 0s 2ms/step - loss: 1.0312 - sparse_categorical_accuracy: 0.5556 - val_loss: 0.6241 - val_sparse_categorical_accuracy: 1.0000

Epoch 00014: val_loss improved from 0.64387 to 0.62407, saving model to models/model2_linear_2.h5
Epoch 15/1000
9/9 [==============================] - 0s 2ms/step - 


Epoch 00044: val_loss did not improve from 0.54382
Epoch 00044: early stopping
====eval=====
['loss', 'sparse_categorical_accuracy']
1/1 [==============================] - 2s 2s/step
[0.5438179969787598, 1.0]
====done=====
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_24 (InputLayer)        (None, 200, 300, 3)       0         
_________________________________________________________________
flatten_24 (Flatten)         (None, 180000)            0         
_________________________________________________________________
dropout_54 (Dropout)         (None, 180000)            0         
_________________________________________________________________
dense_14 (Dense)             (None, 2)                 360002    
Total params: 360,002
Trainable params: 360,002
Non-trainable params: 0
_________________________________________________________________
Train on 9 samples, validate on 1 samples
Ep


Epoch 00005: val_loss did not improve from 0.84566
Epoch 6/1000
9/9 [==============================] - 0s 2ms/step - loss: 1.2514 - sparse_categorical_accuracy: 0.3333 - val_loss: 1.0021 - val_sparse_categorical_accuracy: 0.0000e+00

Epoch 00006: val_loss did not improve from 0.84566
Epoch 7/1000
9/9 [==============================] - 0s 2ms/step - loss: 1.4038 - sparse_categorical_accuracy: 0.4444 - val_loss: 1.0330 - val_sparse_categorical_accuracy: 0.0000e+00

Epoch 00007: val_loss did not improve from 0.84566
Epoch 8/1000
9/9 [==============================] - 0s 2ms/step - loss: 1.2637 - sparse_categorical_accuracy: 0.5556 - val_loss: 1.0646 - val_sparse_categorical_accuracy: 0.0000e+00

Epoch 00008: val_loss did not improve from 0.84566
Epoch 9/1000
9/9 [==============================] - 0s 2ms/step - loss: 1.0254 - sparse_categorical_accuracy: 0.5556 - val_loss: 1.0996 - val_sparse_categorical_accuracy: 0.0000e+00

Epoch 00009: val_loss did not improve from 0.84566
Epoch 10/100


Epoch 00015: val_loss did not improve from 0.01284
Epoch 16/1000
9/9 [==============================] - 0s 3ms/step - loss: 1.4134 - sparse_categorical_accuracy: 0.4444 - val_loss: 0.1537 - val_sparse_categorical_accuracy: 1.0000

Epoch 00016: val_loss did not improve from 0.01284
Epoch 17/1000
9/9 [==============================] - 0s 2ms/step - loss: 0.9612 - sparse_categorical_accuracy: 0.5556 - val_loss: 0.1728 - val_sparse_categorical_accuracy: 1.0000

Epoch 00017: val_loss did not improve from 0.01284
Epoch 18/1000
9/9 [==============================] - 0s 2ms/step - loss: 0.8668 - sparse_categorical_accuracy: 0.6667 - val_loss: 0.1922 - val_sparse_categorical_accuracy: 1.0000

Epoch 00018: val_loss did not improve from 0.01284
Epoch 19/1000
9/9 [==============================] - 0s 2ms/step - loss: 0.8537 - sparse_categorical_accuracy: 0.6667 - val_loss: 0.2117 - val_sparse_categorical_accuracy: 1.0000

Epoch 00019: val_loss did not improve from 0.01284
Epoch 20/1000
9/9 [=====

9/9 [==============================] - 5s 585ms/step - loss: 1.9971 - sparse_categorical_accuracy: 0.4444 - val_loss: 0.2084 - val_sparse_categorical_accuracy: 1.0000

Epoch 00001: val_loss improved from inf to 0.20839, saving model to models/model2_linear_7.h5
Epoch 2/1000
9/9 [==============================] - 0s 2ms/step - loss: 1.9856 - sparse_categorical_accuracy: 0.2222 - val_loss: 0.2548 - val_sparse_categorical_accuracy: 1.0000

Epoch 00002: val_loss did not improve from 0.20839
Epoch 3/1000
9/9 [==============================] - 0s 2ms/step - loss: 1.3485 - sparse_categorical_accuracy: 0.2222 - val_loss: 0.3089 - val_sparse_categorical_accuracy: 1.0000

Epoch 00003: val_loss did not improve from 0.20839
Epoch 4/1000
9/9 [==============================] - 0s 2ms/step - loss: 1.4417 - sparse_categorical_accuracy: 0.4444 - val_loss: 0.3703 - val_sparse_categorical_accuracy: 1.0000

Epoch 00004: val_loss did not improve from 0.20839
Epoch 5/1000
9/9 [==============================


Epoch 00010: val_loss did not improve from 1.83090
Epoch 11/1000
9/9 [==============================] - 0s 2ms/step - loss: 0.6369 - sparse_categorical_accuracy: 0.7778 - val_loss: 2.0867 - val_sparse_categorical_accuracy: 0.0000e+00

Epoch 00011: val_loss did not improve from 1.83090
Epoch 12/1000
9/9 [==============================] - 0s 2ms/step - loss: 0.7715 - sparse_categorical_accuracy: 0.6667 - val_loss: 2.1102 - val_sparse_categorical_accuracy: 0.0000e+00

Epoch 00012: val_loss did not improve from 1.83090
Epoch 13/1000
9/9 [==============================] - 0s 2ms/step - loss: 1.0422 - sparse_categorical_accuracy: 0.4444 - val_loss: 2.1409 - val_sparse_categorical_accuracy: 0.0000e+00

Epoch 00013: val_loss did not improve from 1.83090
Epoch 14/1000
9/9 [==============================] - 0s 3ms/step - loss: 0.7754 - sparse_categorical_accuracy: 0.4444 - val_loss: 2.1773 - val_sparse_categorical_accuracy: 0.0000e+00

Epoch 00014: val_loss did not improve from 1.83090
Epoch 15


Epoch 00018: val_loss did not improve from 3.08283
Epoch 19/1000
9/9 [==============================] - 0s 2ms/step - loss: 0.5100 - sparse_categorical_accuracy: 0.8889 - val_loss: 3.1434 - val_sparse_categorical_accuracy: 0.0000e+00

Epoch 00019: val_loss did not improve from 3.08283
Epoch 20/1000
9/9 [==============================] - 0s 2ms/step - loss: 0.4995 - sparse_categorical_accuracy: 0.7778 - val_loss: 3.1571 - val_sparse_categorical_accuracy: 0.0000e+00

Epoch 00020: val_loss did not improve from 3.08283
Epoch 21/1000
9/9 [==============================] - 0s 2ms/step - loss: 0.5722 - sparse_categorical_accuracy: 0.8889 - val_loss: 3.1866 - val_sparse_categorical_accuracy: 0.0000e+00

Epoch 00021: val_loss did not improve from 3.08283
Epoch 22/1000
9/9 [==============================] - 0s 2ms/step - loss: 0.4690 - sparse_categorical_accuracy: 0.6667 - val_loss: 3.2068 - val_sparse_categorical_accuracy: 0.0000e+00

Epoch 00022: val_loss did not improve from 3.08283
Epoch 23

In [11]:
correct = 0
for i in range(y.shape[0]):
    model_name = os.path.join('models', 'model2_cnn_%s.h5' % (i))
    train_fold = np.asarray([j for j in range(y.shape[0]) if j != i])
    val_fold = np.asarray([i])
    model = vanillacnn(0.5)

    adam = keras.optimizers.Adam(lr=0.001)
    model.compile(optimizer=adam, loss='sparse_categorical_crossentropy', metrics=[metrics.sparse_categorical_accuracy])


    checkpoint = ModelCheckpoint(model_name, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
    stop = EarlyStopping(patience=20, verbose=1)

    callbacks_list = [checkpoint, stop]

    model.summary()
    
    model.fit(x=X[train_fold], 
          y=y[train_fold], 
          batch_size=9, 
          epochs=1000, 
          verbose=1, 
          callbacks=callbacks_list, 
          validation_data=(X[val_fold], y[val_fold]), 
          shuffle=False)
    
    print('====eval=====')
    trained_model = load_model(model_name)
    print(trained_model.metrics_names)
    res = trained_model.evaluate(x=X[val_fold], y=y[val_fold])
    print(res)
    correct += res[1]
    print('====done=====')

print('CNN Model Leave-one-out Cross Validation Accuracy:', correct/y.shape[0])


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_11 (InputLayer)        (None, 200, 300, 3)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 100, 150, 32)      896       
_________________________________________________________________
batch_normalization_1 (Batch (None, 100, 150, 32)      128       
_________________________________________________________________
activation_1 (Activation)    (None, 100, 150, 32)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 50, 75, 32)        0         
_________________________________________________________________
dropout_11 (Dropout)         (None, 50, 75, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 25, 38, 64)        18496     
__________


Epoch 00022: val_loss improved from 0.45692 to 0.43730, saving model to models/model2_cnn_0.h5
Epoch 23/1000
9/9 [==============================] - 0s 2ms/step - loss: 1.3821 - sparse_categorical_accuracy: 0.6667 - val_loss: 0.4493 - val_sparse_categorical_accuracy: 1.0000

Epoch 00023: val_loss did not improve from 0.43730
Epoch 24/1000
9/9 [==============================] - 0s 3ms/step - loss: 0.6128 - sparse_categorical_accuracy: 0.7778 - val_loss: 0.4432 - val_sparse_categorical_accuracy: 1.0000

Epoch 00024: val_loss did not improve from 0.43730
Epoch 25/1000
9/9 [==============================] - 0s 3ms/step - loss: 1.2471 - sparse_categorical_accuracy: 0.5556 - val_loss: 0.4451 - val_sparse_categorical_accuracy: 1.0000

Epoch 00025: val_loss did not improve from 0.43730
Epoch 26/1000
9/9 [==============================] - 0s 3ms/step - loss: 0.1562 - sparse_categorical_accuracy: 1.0000 - val_loss: 0.4322 - val_sparse_categorical_accuracy: 1.0000

Epoch 00026: val_loss improved 


Epoch 00054: val_loss did not improve from 0.17930
Epoch 55/1000
9/9 [==============================] - 0s 3ms/step - loss: 0.2006 - sparse_categorical_accuracy: 0.8889 - val_loss: 0.1919 - val_sparse_categorical_accuracy: 1.0000

Epoch 00055: val_loss did not improve from 0.17930
Epoch 56/1000
9/9 [==============================] - 0s 3ms/step - loss: 0.5981 - sparse_categorical_accuracy: 0.7778 - val_loss: 0.1915 - val_sparse_categorical_accuracy: 1.0000

Epoch 00056: val_loss did not improve from 0.17930
Epoch 57/1000
9/9 [==============================] - 0s 3ms/step - loss: 0.6181 - sparse_categorical_accuracy: 0.7778 - val_loss: 0.1871 - val_sparse_categorical_accuracy: 1.0000

Epoch 00057: val_loss did not improve from 0.17930
Epoch 58/1000
9/9 [==============================] - 0s 3ms/step - loss: 0.3920 - sparse_categorical_accuracy: 0.7778 - val_loss: 0.1704 - val_sparse_categorical_accuracy: 1.0000

Epoch 00058: val_loss improved from 0.17930 to 0.17045, saving model to mod


Epoch 00088: val_loss did not improve from 0.08172
Epoch 89/1000
9/9 [==============================] - 0s 3ms/step - loss: 0.6789 - sparse_categorical_accuracy: 0.6667 - val_loss: 0.1686 - val_sparse_categorical_accuracy: 1.0000

Epoch 00089: val_loss did not improve from 0.08172
Epoch 90/1000
9/9 [==============================] - 0s 3ms/step - loss: 0.6730 - sparse_categorical_accuracy: 0.6667 - val_loss: 0.1902 - val_sparse_categorical_accuracy: 1.0000

Epoch 00090: val_loss did not improve from 0.08172
Epoch 91/1000
9/9 [==============================] - 0s 3ms/step - loss: 1.0015 - sparse_categorical_accuracy: 0.7778 - val_loss: 0.1999 - val_sparse_categorical_accuracy: 1.0000

Epoch 00091: val_loss did not improve from 0.08172
Epoch 92/1000
9/9 [==============================] - 0s 3ms/step - loss: 0.3307 - sparse_categorical_accuracy: 0.7778 - val_loss: 0.1859 - val_sparse_categorical_accuracy: 1.0000

Epoch 00092: val_loss did not improve from 0.08172
Epoch 93/1000
9/9 [=====


Epoch 00007: val_loss did not improve from 1.91294
Epoch 8/1000
9/9 [==============================] - 0s 4ms/step - loss: 0.9622 - sparse_categorical_accuracy: 0.6667 - val_loss: 1.6965 - val_sparse_categorical_accuracy: 0.0000e+00

Epoch 00008: val_loss improved from 1.91294 to 1.69646, saving model to models/model2_cnn_1.h5
Epoch 9/1000
9/9 [==============================] - 0s 3ms/step - loss: 1.6501 - sparse_categorical_accuracy: 0.6667 - val_loss: 1.3907 - val_sparse_categorical_accuracy: 0.0000e+00

Epoch 00009: val_loss improved from 1.69646 to 1.39071, saving model to models/model2_cnn_1.h5
Epoch 10/1000
9/9 [==============================] - 0s 3ms/step - loss: 1.3182 - sparse_categorical_accuracy: 0.5556 - val_loss: 1.1925 - val_sparse_categorical_accuracy: 0.0000e+00

Epoch 00010: val_loss improved from 1.39071 to 1.19253, saving model to models/model2_cnn_1.h5
Epoch 11/1000
9/9 [==============================] - 0s 3ms/step - loss: 1.6420 - sparse_categorical_accuracy: 0.

...


Epoch 00001: val_loss improved from inf to 0.42443, saving model to models/model2_cnn_5.h5
Epoch 2/1000
9/9 [==============================] - 0s 3ms/step - loss: 2.4637 - sparse_categorical_accuracy: 0.5556 - val_loss: 0.4021 - val_sparse_categorical_accuracy: 1.0000

Epoch 00002: val_loss improved from 0.42443 to 0.40208, saving model to models/model2_cnn_5.h5
Epoch 3/1000
9/9 [==============================] - 0s 3ms/step - loss: 1.6855 - sparse_categorical_accuracy: 0.7778 - val_loss: 0.4268 - val_sparse_categorical_accuracy: 1.0000

Epoch 00003: val_loss did not improve from 0.40208
Epoch 4/1000
9/9 [==============================] - 0s 3ms/step - loss: 2.2357 - sparse_categorical_accuracy: 0.5556 - val_loss: 0.3292 - val_sparse_categorical_accuracy: 1.0000

Epoch 00004: val_loss improved from 0.40208 to 0.32922, saving model to models/model2_cnn_5.h5
Epoch 5/1000
9/9 [==============================] - 0s 3ms/step - loss: 1.8077 - sparse_categorical_accuracy: 0.6667 - val_loss: 0

Train on 9 samples, validate on 1 samples
Epoch 1/1000
9/9 [==============================] - 3s 319ms/step - loss: 2.1059 - sparse_categorical_accuracy: 0.4444 - val_loss: 0.3963 - val_sparse_categorical_accuracy: 1.0000

Epoch 00001: val_loss improved from inf to 0.39630, saving model to models/model2_cnn_6.h5
Epoch 2/1000
9/9 [==============================] - 0s 3ms/step - loss: 0.4154 - sparse_categorical_accuracy: 0.7778 - val_loss: 0.7284 - val_sparse_categorical_accuracy: 0.0000e+00

Epoch 00002: val_loss did not improve from 0.39630
Epoch 3/1000
9/9 [==============================] - 0s 4ms/step - loss: 1.5377 - sparse_categorical_accuracy: 0.6667 - val_loss: 1.1715 - val_sparse_categorical_accuracy: 0.0000e+00

Epoch 00003: val_loss did not improve from 0.39630
Epoch 4/1000
9/9 [==============================] - 0s 3ms/step - loss: 2.5385 - sparse_categorical_accuracy: 0.5556 - val_loss: 1.4760 - val_sparse_categorical_accuracy: 0.0000e+00

Epoch 00004: val_loss did not impro


Epoch 00001: val_loss improved from inf to 1.19930, saving model to models/model2_cnn_7.h5
Epoch 2/1000
9/9 [==============================] - 0s 3ms/step - loss: 0.8809 - sparse_categorical_accuracy: 0.6667 - val_loss: 2.3645 - val_sparse_categorical_accuracy: 0.0000e+00

Epoch 00002: val_loss did not improve from 1.19930
Epoch 3/1000
9/9 [==============================] - 0s 4ms/step - loss: 0.8702 - sparse_categorical_accuracy: 0.7778 - val_loss: 3.0203 - val_sparse_categorical_accuracy: 0.0000e+00

Epoch 00003: val_loss did not improve from 1.19930
Epoch 4/1000
9/9 [==============================] - 0s 3ms/step - loss: 2.3407 - sparse_categorical_accuracy: 0.5556 - val_loss: 3.3990 - val_sparse_categorical_accuracy: 0.0000e+00

Epoch 00004: val_loss did not improve from 1.19930
Epoch 5/1000
9/9 [==============================] - 0s 3ms/step - loss: 0.9873 - sparse_categorical_accuracy: 0.7778 - val_loss: 3.3731 - val_sparse_categorical_accuracy: 0.0000e+00

Epoch 00005: val_loss d


Epoch 00002: val_loss improved from 0.77601 to 0.41735, saving model to models/model2_cnn_8.h5
Epoch 3/1000
9/9 [==============================] - 0s 3ms/step - loss: 1.9361 - sparse_categorical_accuracy: 0.5556 - val_loss: 0.2853 - val_sparse_categorical_accuracy: 1.0000

Epoch 00003: val_loss improved from 0.41735 to 0.28531, saving model to models/model2_cnn_8.h5
Epoch 4/1000
9/9 [==============================] - 0s 3ms/step - loss: 2.6705 - sparse_categorical_accuracy: 0.3333 - val_loss: 0.3048 - val_sparse_categorical_accuracy: 1.0000

Epoch 00004: val_loss did not improve from 0.28531
Epoch 5/1000
9/9 [==============================] - 0s 3ms/step - loss: 2.3253 - sparse_categorical_accuracy: 0.2222 - val_loss: 0.3601 - val_sparse_categorical_accuracy: 1.0000

Epoch 00005: val_loss did not improve from 0.28531
Epoch 6/1000
9/9 [==============================] - 0s 3ms/step - loss: 0.6885 - sparse_categorical_accuracy: 0.5556 - val_loss: 0.4140 - val_sparse_categorical_accuracy:


Epoch 00001: val_loss improved from inf to 0.42328, saving model to models/model2_cnn_9.h5
Epoch 2/1000
9/9 [==============================] - 0s 3ms/step - loss: 1.0058 - sparse_categorical_accuracy: 0.4444 - val_loss: 0.5475 - val_sparse_categorical_accuracy: 1.0000

Epoch 00002: val_loss did not improve from 0.42328
Epoch 3/1000
9/9 [==============================] - 0s 3ms/step - loss: 1.1144 - sparse_categorical_accuracy: 0.7778 - val_loss: 0.5644 - val_sparse_categorical_accuracy: 1.0000

Epoch 00003: val_loss did not improve from 0.42328
Epoch 4/1000
9/9 [==============================] - 0s 4ms/step - loss: 3.1055 - sparse_categorical_accuracy: 0.3333 - val_loss: 0.6243 - val_sparse_categorical_accuracy: 1.0000

Epoch 00004: val_loss did not improve from 0.42328
Epoch 5/1000
9/9 [==============================] - 0s 4ms/step - loss: 0.7642 - sparse_categorical_accuracy: 0.6667 - val_loss: 0.6275 - val_sparse_categorical_accuracy: 1.0000

Epoch 00005: val_loss did not improve f